In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait

import time
from os.path  import basename
import requests
from PIL import Image
from IPython.display import display
import pandas as pd

In [36]:
PATH = 'C:\Program Files (x86)\chromedriver.exe'

In [2]:
def features_to_df(input_string):
    features_df = pd.DataFrame(columns=['zametr', 'pokoje', 'powierzchnia', 'umeblowane', 'czynsz', 'rodzaj zabudowy', 'rynek', 'piętro', 'typ', 'forma wlasnosci', \
                                    'stan wykonczenia','balkon/ogrod/taras', 'miejsce parkingowe', 'ogrzewanie'])
    possible_feaures = ['zametr', 'pokoje', 'powierzchnia', 'umeblowane', 'czynsz', 'rodzaj zabudowy', 'rynek', 'piętro', 'typ', 'forma wlasnosci', \
                                    'stan wykonczenia','balkon/ogrod/taras', 'miejsce parkingowe', 'ogrzewanie']


    pairs = [pair.strip().split(':') for pair in input_string.rstrip(',').split(', ')]
    data_dict = {key.strip(): value.strip() for key, value in pairs}
    
    if 'Cena za m²' in data_dict.keys():
        data_dict['zametr'] = data_dict.pop('Cena za m²')
    if 'Poziom' in data_dict.keys():
        data_dict['Piętro'] = data_dict.pop('Poziom')
    if 'Liczba pokoi' in data_dict.keys():
        data_dict['pokoje'] = data_dict.pop('Liczba pokoi').strip(" ")[0]

    data_dict_lower = {key.lower(): value for key, value in data_dict.items()}

    for feature in possible_feaures:
        if feature not in data_dict_lower.keys():
            data_dict_lower[feature] = None
    features_df.loc[len(features_df)] = data_dict_lower
    return features_df

***OLX***

In [3]:
def get_info_olx(url):
    
    driver = webdriver.Chrome(PATH)
    driver.get(url)
    driver.implicitly_wait(1)
    button = driver.find_element(By.ID, "onetrust-accept-btn-handler")
    button.click()
    
    nazwa = driver.find_element(By.CLASS_NAME, "css-77x51t").text
    cena = driver.find_element(By.CLASS_NAME, "css-93ez2t").text
    opis = driver.find_element(By.CLASS_NAME, "css-1m8mzwg").text.split("OPIS")[1].strip()
    lokalizacja = driver.find_element(By.XPATH, "//div[@id='hydrate-root']/div[@class='css-1ek5um8']/div[@class='css-118kolg']/div[@class='css-1d90tha']/div[@class='css-6u8zs6']/div[@class='css-1dp6pbg']/div[@class='css-1q7h1ph']/section[@class='css-16sja3n']/div[@class='css-13l8eec']/div/p[@class='css-1cju8pu er34gjf0']").text
    tagi = driver.find_elements(By.CLASS_NAME, "css-1r0si1e")
    
    tags = 'typ: '
    for tag in tagi:
        tags = tags + tag.text + ', '

    
    col=['nazwa_ogloszenia', 'cena' ,'opis' ,'lokalizacja']
    dane=[[nazwa,cena, opis, lokalizacja]]
    czesc = pd.DataFrame(data=dane,columns=col)
    kawalek = features_to_df(tags.strip())
    koncowy = pd.concat([czesc, kawalek], ignore_index = False ,axis=1)
    return koncowy
    

***Otodom***

In [4]:
def get_info_otodom(url):
    
    driver = webdriver.Chrome(PATH)
    driver.get(url)
    driver.implicitly_wait(1)
    button = driver.find_element(By.ID, "onetrust-accept-btn-handler")
    button.click()
    driver.maximize_window()
    
    nazwa = driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div[2]/header/h1').text
    cena = driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div[2]/header/strong').text

    
    try:
        zametr = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/header/div[3]').text
    except:
        zametr = None

    try:
        pokoje = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[2]/div/div[3]/div[2]/div/a').text
    except:
        pokoje = None
    
    try:
        lokalizacja = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/header/div[2]/a').text
    except:
        lokalizacja = None
    
    try:
        powierzchnia = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[2]/div/div[1]/div[2]/div').text
    except:
        powierzchnia = None
    
    try:
        umeblowane = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[6]/div/div[10]/div[2]/div').text   # na otodom podpisane jako wyposażenie
    except:
        umeblowane = None
        
    try:
        czynsz = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[2]/div/div[7]/div[2]/div').text
    except:
        czynsz = None
        
    try:
        rodzaj_zabudowy = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[6]/div/div[5]/div[2]/div').text
    except:
        rodzaj_zabudowy  = None
    
    try:
        rynek = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[5]/div/div[1]/div[2]/div').text
    except:
        rynek  = None
    
    try:
        pietro = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[2]/div/div[5]/div[2]/div').text
    except:
        pietro  = None

    try:
        typ = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[6]/div/div[2]/div[2]/div').text   # typ oferenta
    except:
        typ = None
    
    try:
        forma_wlasnosci = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[2]/div/div[2]/div[2]/div').text 
    except:
        forma_wlasnosci = None

    try:
        stan_wykonczenia = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[2]/div/div[4]/div[2]/div').text 
    except:
        stan_wykonczenia = None

    try:
        balkon_ogrod_taras = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[2]/div/div[6]/div[2]/div').text 
    except:
        balkon_ogrod_taras = None

    try:
        miejsce_parkingowe = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[2]/div/div[8]/div[2]/span/span/div').text 
    except:
        miejsce_parkingowe = None

    try:
        ogrzewanie = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/div[2]/div/div[10]/div[2]/div').text 
    except:
        ogrzewanie = None

    try:
        button = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/section[2]/div/button')
        button.click()
    except:
        pass

    try:
        opis = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/section[2]/div/div/div').text
    except:
        opis = None
    if opis == 'Obsługa zdalna':
        opis = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[2]/section[3]/div/div/div').text
        
    col = ['nazwa_ogloszenia', 'cena', 'opis', 'lokalizacja','zametr','pokoje', 'powierzchnia' , 'umeblowane', 'czynsz' , 'rodzaj zabudowy', 'rynek', 'piętro', 'typ', 'forma wlasnosci','stan wykonczenia', 'balkon/ogrod/taras', 'miejsce parkingowe', 'ogrzewanie']
    dane = [[nazwa, cena, opis, lokalizacja,zametr,pokoje, powierzchnia, umeblowane, czynsz, rodzaj_zabudowy, rynek, pietro, typ, forma_wlasnosci, stan_wykonczenia, balkon_ogrod_taras, miejsce_parkingowe, ogrzewanie]]

    

    

    return pd.DataFrame(data=dane,columns=col)

***Funkcja do wyciagania linkow z olx***

In [7]:
def get_links(url, strony):
    links = []
    for i in range (1,strony+1):
        urls = url  + '?page=' + str(i)
        driver = webdriver.Chrome(PATH)
        driver.get(url)
        driver.maximize_window()
        button = driver.find_element(By.ID, "onetrust-accept-btn-handler")
        button.click()
        linki = driver.find_elements(By.CLASS_NAME, "css-rc5s2u")
        
        for link in linki:
            links.append(link.get_attribute("href"))
            
    return links

In [9]:
linki_do_scrapowania_olx = get_links('https://www.olx.pl/nieruchomosci/mieszkania/sprzedaz/wroclaw/',1) #TU ZMIENIC LICZBE STRON!!!!!!!!

In [71]:
def get_info(linki):
    
    wyniki = pd.DataFrame()
    odrzucone = []
    
    for link in linki:
        if 'otodom' in link:
            try:
                wyniki = pd.concat([wyniki, get_info_otodom(link)], ignore_index = True)
            except:
                odrzucone.append(link)
        if 'olx' in link:
            try:
                wyniki = pd.concat([wyniki, get_info_olx(link)], ignore_index = True)
            except:
                odrzucone.append(link)
    
    wyniki.to_csv('D:/Skrypty/IX semestr/MM w Pythonie/Projekt/oferty.csv', sep = ',', index = False, encoding = 'utf-8') 
    
    return odrzucone

In [ ]:
#Koncowo dane scrapujemy z tego: linki_do_scrapowania_olx TYLKO TAM DAC ODPOWIEDNIA LICZBE STRON A NIE 3!!!!

In [ ]:
# 3 przykladowe linki, dwa pierwsze otodom trzeci olx

In [73]:
get_info(['https://www.otodom.pl/pl/oferta/jagodno-lux-3pok-garaz-taras-52m-2-lazienki-ID4oxRA.html?_gl=1*1bp7fjf*_ga*MjkxMDI1MjA2LjE3MDQ5MTI1OTk.*_ga_6PZTQNYS5C*MTcwNTMxMjYwOC45LjEuMTcwNTMxMzk0NC41MS4wLjA.&_ga=2.24430653.640981245.1705246096-291025206.1704912599','https://www.otodom.pl/pl/oferta/rezerwacja-3m-duzy-taras-na-najwyzszym-pietrze-ID4ozaq.html?_gl=1*1bp7fjf*_ga*MjkxMDI1MjA2LjE3MDQ5MTI1OTk.*_ga_6PZTQNYS5C*MTcwNTMxMjYwOC45LjEuMTcwNTMxMzk0NC41MS4wLjA.&_ga=2.24430653.640981245.1705246096-291025206.1704912599','https://www.olx.pl/d/oferta/mieszkanie-muchobor-wielki-CID3-IDYgrOg.html'])

[]

In [ ]:
get_info(linki_do_scrapowania_olx)